https://www.kaggle.com/code/mcarujo/synthetic-data-generation-sdv-tutotial

In [ ]:
!pip install sdv==0.13.1 -q


In [ ]:
import warnings

import pandas as pd

from sdv.evaluation import evaluate
from sdv.metrics.tabular import CSTest, KSTest, LogisticDetection, SVCDetection
from sdv.tabular import CTGAN, TVAE, CopulaGAN, GaussianCopula

warnings.filterwarnings("ignore")


In [ ]:
import numpy as np
real_data = pd.read_csv("/content/PS1B_train.csv")

print(real_data.shape)
real_data.head()


(307, 9)


,C1,C2,C3,C4,C5,C6,C7,C8,Outcome
0,5,187,76,27,207,43.6,1.034,53,1
1,3,129,64,29,115,26.4,0.219,28,1
2,0,131,0,0,0,43.2,0.270,26,1
3,10,115,0,0,0,0.0,0.261,30,1
4,0,107,60,25,0,26.4,0.133,23,0


In [ ]:
data=real_data.copy()
X = data.iloc[:, :-1]
y=data.iloc[:, -1]

In [ ]:
no_ones=0
no_zeros=0
for i in list(y):
  if i==1:
    no_ones=no_ones+1
  else:
    no_zeros=no_zeros+1
print(no_ones,no_zeros)

94 213


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
Xs, ys = sm.fit_resample(X, y.ravel())

print('After OverSampling, the shape of train_X: {}'.format(Xs.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(ys.shape))

print("After OverSampling, counts of label '1': {}".format(sum(ys == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(ys == 0)))

After OverSampling, the shape of train_X: (426, 8)
After OverSampling, the shape of train_y: (426,) 

After OverSampling, counts of label '1': 213
After OverSampling, counts of label '0': 213


In [ ]:
frames = [Xs, pd.DataFrame(ys)]
  
result = pd.concat(frames,axis=1, join='inner')
result.rename(columns = {0:'Target'}, inplace = True)

In [ ]:
pd.pandas.set_option('display.max_columns', None)

result.head(15)

,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,5,187,76,27,207,43.6,1.034,53,1
1,3,129,64,29,115,26.4,0.219,28,1
2,0,131,0,0,0,43.2,0.270,26,1
3,10,115,0,0,0,0.0,0.261,30,1
4,0,107,60,25,0,26.4,0.133,23,0
5,5,99,54,28,83,34.0,0.499,30,0
6,9,152,78,34,171,34.2,0.893,33,1
7,10,108,66,0,0,32.4,0.272,42,1
8,0,165,76,43,255,47.9,0.259,26,0
9,1,135,54,0,0,26.7,0.687,62,0


In [ ]:
model_tvae=TVAE()

In [ ]:
model_tvae.fit(result)

In [ ]:
real_data_test = pd.read_csv("/content/PS1B_test.csv")

In [ ]:
new_data_tvae = model_tvae.sample(real_data_test.shape[0])

In [ ]:
data=new_data_tvae.copy()
X = data.iloc[:, :-1]
y=data.iloc[:, -1]
X

,C1,C2,C3,C4,C5,C6,C7,C8
0,5,138,75,29,0,33.076560,0.295104,54
1,6,118,74,38,5,32.536844,0.531592,49
2,9,116,80,1,0,27.221998,0.294216,61
3,7,134,70,30,0,32.585858,0.700446,25
4,0,123,83,15,1,27.578308,0.202838,25
...,...,...,...,...,...,...,...,...
456,9,126,72,29,3,33.334703,0.089512,25
457,0,122,62,27,10,29.672535,0.254924,24
458,11,125,79,35,13,33.287583,0.759168,40
459,7,97,71,0,0,25.461337,0.084000,40


In [ ]:
sum(y==1)

245

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 10)
classifier.fit(X, y)

LogisticRegression(random_state=10)

In [ ]:
real_data_test.drop(['Index'], axis=1, inplace=True)


KeyError: ignored

In [ ]:
real_data_test.head()

,C1,C2,C3,C4,C5,C6,C7,C8
0,8,95,72,0,0,36.8,0.485,57
1,1,93,56,11,0,22.5,0.417,22
2,6,154,78,41,140,46.1,0.571,27
3,3,84,72,32,0,37.2,0.267,28
4,3,99,62,19,74,21.8,0.279,26


In [ ]:
y_pred_lr = classifier.predict(real_data_test)
y_pred=pd.DataFrame(y_pred_lr)

In [28]:
y_pred

,0
0,1
1,0
2,1
3,0
4,0
...,...
456,1
457,1
458,0
459,1


In [29]:
y_pred.to_csv("1B_sub.csv")

In [ ]:

from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test_lr, y_pred_lr))

Accuracy :  0.7781333333333333


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_lr,y_pred_lr))

              precision    recall  f1-score   support

           0       0.78      0.79      0.78      7667
           1       0.78      0.77      0.77      7333

    accuracy                           0.78     15000
   macro avg       0.78      0.78      0.78     15000
weighted avg       0.78      0.78      0.78     15000



In [ ]:

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestClassifier
 
clf = RandomForestClassifier(n_estimators = 100)  
  
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train_lr, y_train_lr)
  
# performing predictions on the test dataset
y_pred = clf.predict(X_test_lr)

In [ ]:
print(classification_report(y_test_lr,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81      7667
           1       0.81      0.79      0.80      7333

    accuracy                           0.81     15000
   macro avg       0.81      0.81      0.81     15000
weighted avg       0.81      0.81      0.81     15000



In [ ]:
print ("Accuracy : ", accuracy_score(y_test_lr, y_pred))

Accuracy :  0.8057333333333333
